In [ ]:
import numpy as np
import pandas as pd

np.set_printoptions(precision=3, linewidth=150)


xsL = [(0,1),(0,1),]
qs  = (0,1,2)

if False:
    ys  = ((0,0), (0,1), (1,0), (1,1))

    pX0X1 = np.array([[1, 0.1],[0.2,1]]).astype('float')
    #pX0X1 = np.eye(2)
    pX0X1 /= pX0X1.sum()

    pXY = {}
    for xix, xs in enumerate(xsL):
        pXY[xix] = np.zeros((len(xs),len(ys)))
        for xndx, x in enumerate(xs):
            for yndx, y in enumerate(ys):
                if y[xix] == x:
                    pXY[xix][xndx,yndx] = pX0X1[y[0],y[1]]
                    #print('h x%d=%d y=%s'%(xix, x, y),  y[0],y[1])
        pXY[xix]/=pXY[xix].sum()

    pY = pXY[0].sum(axis=0)
    # print(pY)
    assert(np.allclose(pY, pXY[1].sum(axis=0)))

else:
    ys  = (0,1,2)
    
    pYX0X1 = np.random.random((len(ys),len(xsL[0]),len(xsL[1])))
    pYX0X1 /= pYX0X1.sum()
    
    pY = pYX0X1.sum(axis=-1).sum(axis=-1)
    pXY= {}
    pXY[0] = pYX0X1.sum(axis=-1).T
    pXY[1] = pYX0X1.sum(axis=1).T
assert(np.allclose(pY, pXY[0].sum(axis=0)))
assert(np.allclose(pY, pXY[1].sum(axis=0)))


In [ ]:
ixs = {}
names = {}
for q in qs[:-1]:
    for y in ys:
        k= ('q|y',q,y)
        ixs[k] = len(ixs)
        names[k] = 'p(Q=%d|Y=%d)' % (q,y)
for xix, xs in enumerate(xsL):
    for q in qs[:-1]:
        for x in xs:
            k = ('q|x%d'%xix,q,x)
            ixs[k] = len(ixs)
            names[k] = 'p(Q=%d|X%d=%d)' % (q,xix,x)
N = len(ixs)

In [ ]:
A = []
for q in qs[:-1]:
    for xix, xs in enumerate(xsL):
        for x in xs:
            r = np.zeros(N)
            r[ixs[('q|x%d'%xix,q,x)]] = 1
            A.append(r)
            

for xix, xs in enumerate(xsL):
    for x in xs:
        r = np.zeros(N)
        for q in qs[:-1]:
            r[ixs[('q|x%d'%xix,q,x)]] = 1
        A.append(r)

B = []

for xix, xs in enumerate(xsL):
    for q in qs:
        for yndx, y in enumerate(ys):
            r = np.zeros(N)
            #r = np.zeros(N).astype('str')
            cval = -pY[yndx]
            #cval = 'pY(%d)'%yndx
            if q == qs[-1]:
                for q2 in qs[:-1]:
                    r[ixs[('q|y',q2,y)]] = cval
            else:
                cval *= -1
                # cval = '-'+cval
                r[ixs[('q|y',q,y)]] = cval
                
            for xndx, x in enumerate(xs):
                cval = pXY[xix][xndx,yndx]
                if q == qs[-1]:
                    for q2 in qs[:-1]:
                        r[ixs[('q|x%d'%xix,q2,x)]] = cval
                else:
                        r[ixs[('q|x%d'%xix,q,x)]] = cval
            B.append(r)
            
Amx = np.array(A)
Bmx = np.array(B)
Cmx = np.vstack([A,B])
print('shape(A) = %s, rank(A) = %d' % (Amx.shape,np.linalg.matrix_rank(Amx)))
print('shape(B) = %s, rank(B) = %d' % (Bmx.shape,np.linalg.matrix_rank(Bmx)))
print('shape(C) = %s, rank(C) = %d' % (Cmx.shape,np.linalg.matrix_rank(Cmx)))

cols = ['-',]*len(ixs)
for k,v in ixs.items():
    cols[v] = names[k]
df = pd.DataFrame(Bmx.copy(), columns=cols)
df[df==0] = '.'
df


In [ ]:
print(Bmx[:6].sum(axis=0))

fit = np.linalg.lstsq(Bmx[:10].T, Bmx[11])
print(fit[1])
fit[0][np.isclose(fit[0],0)]=0
print(fit[0])

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
plt.imshow(Bmx)
plt.show()